# Parkinson's disease progression notebook

### Motivatie en originaliteit van de gebruikte methoden


Het voorspellen van het verloop van de ziekte van Parkinson (PD) op basis van eiwitten uit onze gegeven is een complexe taak die een doordachte aanpak vereist. Hier zijn de stappen en overwegingen die wij hebben doorlopen voor het kiezen van het juiste model.

#### Data Preprocessing:

Wij zijn begonnen met het opgschonenen en voorbereiden van onze data, Hierbij hebben wij processen gebruikte als groeperen, piviton en mergen.


In [ ]:
def prepare_dataset(train_proteins: pd.DataFrame, train_peptides: pd.DataFrame):

    # Grouping 
    df_protein_grouped = train_proteins.groupby(["visit_id","UniProt"])["NPX"].mean().reset_index()
    df_peptide_grouped = train_peptides.groupby(["visit_id","Peptide"])["PeptideAbundance"].mean().reset_index()

    # Pivoting
    df_protein = df_protein_grouped.pivot(index="visit_id", columns="UniProt", values="NPX").rename_axis(columns=None).reset_index()
    df_peptide = df_peptide_grouped.pivot(index="visit_id", columns="Peptide", values="PeptideAbundance").rename_axis(columns=None).reset_index()
    
    # Merging
    pro_pep_df = df_protein.merge(df_peptide, on=["visit_id"], how="left")
    
    return pro_pep_df

#### Feature Selection:

Wij hebben ervoor gekozen om alle features te gebruiken die mogelijk zijn, dit omdat wij niet veel verschil konden vinden tussen features, dus het weghouden van bepaalde features zou in dat geval ononderbouwd zijn.

#### Model Selection:

Wij hebben ervoor gekozen om zowel met een Random Forest als Neural Network onze target te predicten, hieronder onze argumentatie waarom deze modellen goed bij onze doel past.

#### Random Forests:

- Random Forests is een leermethode die meerdere beslissingsbomen combineert om voorspellende nauwkeurigheid te verbeteren en overfitting te beheersen. het kan goed omgaan met data met veel dimensies, waardoor het geschikt is voor datasets met een groot aantal kenmerken van in dit geval eiwitten en petide waardes.
- Robust tegen overfitting, outliers, noise,  and missende waardes.
- kan niet-lineaire relaties in de gegevens vastleggen, waardoor het geschikt is in scenario's waarin de relatie tussen de waarden van de eiwiten en de ziekteprogressie non linear is.

#### Neural Network:

- Goed voor het vastleggen van complexe patronen, vooral in grote datasets.
- Neurale netwerken kunnen automatisch relevante kenmerken uit de data halen, die wij met onze huidige domein kennis niet zouden herkennen.
- Ze zijn flexibel en kunnen worden aangepast aan verschillende soorten gegevens en probleemscenario's. dit zal ons voornamelijk helpen omdat wij met dit probleem meerdere targets hebben.


#### Training models:

Wij hebben ervoor gekozen 20/80 training split te gebruiken voor beide modellen



In [ ]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

### Random Forest

In [ ]:
rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION, verbose=0)
rf.compile(metrics=["mse"])

rf.fit(x=train_ds)

#### Evaluating Random Forest model


In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()
evaluation = rf.evaluate(x=test_ds)

#### Het Resultaat van dit model:

- Updrs 1 mse: 19.5633
- Updrs 2 mse: 27.9235
- Updrs 3 mse: 191.5821
- Updrs 4 mse: 7.7804

### Neural Network

In [ ]:
# Define hidden layers and output layers
model = Sequential(
[
    layers.Dense(64, activation="tanh", name="layer1"),   
    layers.Dense(64, activation="tanh", name="layer2"),   
    layers.Dense(64, activation="tanh", name="layer3"),   
    layers.Dense(64, activation="tanh", name="layer4"),   
    layers.Dense(1),
])
    
# Compile model to set optimizer, loss function and metrics
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.MeanSquaredError().name,
    metrics=[losses.MeanSquaredError().name])

# Fit The model
model.fit(
    x_train,
    y_train,
    epochs=10, 
    validation_data=(x_test, y_test)
    )

#### Evaluating Neural Network


In [ ]:
# Evaluate model performance in MeanSquaredError
print(model.evaluate(x_test, y_test))

#### Het Resultaat van dit model:

- Updrs 1 mse: 26.8737
- Updrs 2 mse: 33.0623
- Updrs 3 mse: 237.4834
- Updrs 4 mse: 8.4585

### Kaggle Score

Wij hebben er ook ervoor gekozen om de kaggle scores te vergelijken:

#### Neural network:
- Updrs 1 smape: 65.982
- Updrs 2 smape: 87.542
- Updrs 3 smape: 96.292
- Updrs 4 smape: 148.666

#### Random Forest:
- Updrs 1 smape: 65.982
- Updrs 2 smape: 87.542
- Updrs 3 smape: 96.292
- Updrs 4 smape: 148.666


### Bevindingen

Door zowel de Mean Square Errror als kaggle's Symmetric mean absolute percentage error score te vergelijken hebben wij concluderen dat de Random forest model beter resultaat levert voor ons probleem. 

Uit dit project is gebleken dat alhoewel de Neural network een goede model is hiervoor dat de random forest zowel consistenter als betere voorspellingen gaf. Hierom hebben wij gekozen om dit model in te leveren ondanks dat wij beide modellen uitvoerig getest hebben.